In [1]:
import numpy as np
import pandas as pd
import json
import re

In [2]:
# load json as dict
with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_1.json') as json_file:
    dict_1 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_2.json') as json_file:
    dict_2 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_3.json') as json_file:
    dict_3 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_4.json') as json_file:
    dict_4 = json.load(json_file)

all_records = dict_1 + dict_2 + dict_3 + dict_4

# delete some key
for i in range(len(all_records)):
    del all_records[i]['url'], all_records[i]['title'], all_records[i]['description'], all_records[i]['images']

In [10]:
df_raw = pd.DataFrame(all_records).fillna(value=np.nan)
df_raw = df_raw.drop(['distance'], axis=1)
df_raw

,address,bedroom,toilet,investor,acreage,price,lat,long
0,"dự án the matrix one, đường lê quang đạo, phườ...",2 pn,2 phòng,công ty cổ phần đầu tư mai linh,88 m²,thỏa thuận,21.009386,105.773872
1,"dự án starcity lê văn lương, đường lê văn lươn...",3 pn,2 phòng,ocean group - tập đoàn đại dương,111.7 m²,~32.2 triệu/m²,21.006182,105.806427
2,"dự án vinhomes ocean park gia lâm, xã đa tốn, ...",2 pn,2 phòng,tập đoàn vingroup,68.4 m²,~27.7 triệu/m²,20.996325,105.932869
3,"phố lạc trung, phường vĩnh tuy, hai bà trưng,...",2 pn,2 phòng,NaN,65 m²,~29.2 triệu/m²,21.002986,105.865675
4,"dự án the matrix one, đường lê quang đạo, phườ...",2 pn,2 phòng,công ty cổ phần đầu tư mai linh,88 m²,thỏa thuận,21.009386,105.773872
...,...,...,...,...,...,...,...,...
16395,"rose town, 79, đường ngọc hồi, phường hoàng li...",2 pn,2 phòng,NaN,71 m²,~27.5 triệu/m²,20.956589,105.844887
16396,"dự án park hill premium - times city, đường mi...",2 pn,2 phòng,tập đoàn vingroup,78 m²,~44.9 triệu/m²,20.996578,105.865196
16397,"dự án le grand jardin sài đồng, đường huỳnh vă...",2 pn,2 phòng,công ty cp đầu tư xây dựng số 9 hà nội,54 m²,thỏa thuận,21.035978,105.906296
16398,"dự án việt hưng, phường giang biên, long biên,...",2 pn,NaN,tổng công ty đầu tư phát triển nhà và đô thị hud,85 m²,~30.6 triệu/m²,21.062691,105.908333


In [12]:
def split_number(data):
    try:
        return re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0]
    except:
        return np.nan

def extract_district(data):
    districts = ['Ba Đình', 'Hoàn Kiếm', 'Tây Hồ', 'Long Biên', 'Cầu Giấy', 'Đống Đa', 'Hai Bà Trưng', 'Hoàng Mai', 
                 'Thanh Xuân', 'Sóc Sơn', 'Đông Anh', 'Gia Lâm', 'Nam Từ Liêm', 'Thanh Trì', 'Bắc Từ Liêm', 'Mê Linh', 
                 'Hà Đông', 'Sơn Tây', 'Ba Vì', 'Phúc Thọ', 'Đan Phượng', 'Hoài Đức', 'Quốc Oai', 'Thạch Thất', 
                 'Chương Mỹ', 'Thanh Oai', 'Thường Tín', 'Phú Xuyên', 'Ứng Hòa', 'Mỹ Đức']
    
    for district in districts:
        if district.lower() in data:
            return district
    return np.nan

def extract_million(data):
    if 'triệu' in data:
        try:
            return float(re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0])
        except:
            return 0
    return 0

def extract_billion(data):
    if 'tỷ' in data:
        try:
            return float(re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0])*1000
        except:
            return 0
    return 0

def extract_price(df):
    df['million'] = df['price'].apply(extract_million).astype(np.float64)
    df['billion'] = df['price'].apply(extract_billion).astype(np.float64)
    df['billion_to_million'] = df['billion'] / df['acreage']
    
    df['price'] = df['billion_to_million'] + df['million']
    df['price'] = df['price'].replace(0, np.nan)
    
    return df.drop(['million', 'billion', 'billion_to_million'], axis=1)

    
def process_data(df):
    # extract bedroom, toilet, acreage to number values
    df['bedroom'] = df['bedroom'].apply(split_number).astype(np.float64)
    df['toilet'] = df['toilet'].apply(split_number).astype(np.float64)
    df['acreage'] = df['acreage'].apply(split_number).astype(np.float64)

    # extract address to district values
    df['address'] = df['address'].apply(extract_district)
    
    # drop nan value of columns address, acreage, bedroom
    df = df.dropna(subset=['address', 'acreage', 'bedroom'])
    
    # split price million and billion
    df = extract_price(df)
    
    return df

In [13]:
# dicts to dataframe and replace none to np.nan
df = pd.DataFrame(all_records).fillna(value=np.nan)
df = process_data(df)

/var/folders/hw/wb3xxs5j60jc2wpr6vg_98240000gn/T/ipykernel_33036/557037098.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['million'] = df['price'].apply(extract_million).astype(np.float64)
/var/folders/hw/wb3xxs5j60jc2wpr6vg_98240000gn/T/ipykernel_33036/557037098.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['billion'] = df['price'].apply(extract_billion).astype(np.float64)
/var/folders/hw/wb3xxs5j60jc2wpr6vg_98240000gn/T/ipykernel_33036/557037098.py:37: SettingWithCopyWarning: 
A value i

In [14]:
df

,address,bedroom,toilet,investor,acreage,price,lat,long,distance
0,Nam Từ Liêm,2.0,2.0,công ty cổ phần đầu tư mai linh,88.0,NaN,21.009386,105.773872,8.453731
1,Thanh Xuân,3.0,2.0,ocean group - tập đoàn đại dương,111.7,32.20000,21.006182,105.806427,5.413824
2,Gia Lâm,2.0,2.0,tập đoàn vingroup,68.4,27.70000,20.996325,105.932869,9.104289
3,Hai Bà Trưng,2.0,2.0,NaN,65.0,29.20000,21.002986,105.865675,3.191921
4,Nam Từ Liêm,2.0,2.0,công ty cổ phần đầu tư mai linh,88.0,NaN,21.009386,105.773872,8.453731
...,...,...,...,...,...,...,...,...,...
16395,Hoàng Mai,2.0,2.0,NaN,71.0,27.50000,20.956589,105.844887,8.087054
16396,Hai Bà Trưng,2.0,2.0,tập đoàn vingroup,78.0,44.90000,20.996578,105.865196,3.831080
16397,Long Biên,2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,NaN,21.035978,105.906296,5.644863
16398,Long Biên,2.0,NaN,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.60000,21.062691,105.908333,6.914078


In [16]:
with open('data-to-db.json', 'w', encoding='utf-8') as file:
    json.dump(js, file, ensure_ascii=False)

In [24]:
# read old data and concat new data to total data
old_df = pd.read_csv('old-data.csv')
total_data = pd.concat([old_df, df], axis=0).reset_index()

In [25]:
# replace len(investor) < 30 to 'other'
mask_investor = total_data.investor.map(total_data.investor.value_counts()) < 30
total_data.investor =  total_data.investor.mask(mask_investor, 'other')
total_data.investor = total_data.investor.replace(np.nan, 'other')

In [35]:
# drop record has address is 'Thạch Thất' or 'Mê Linh'
total_data = total_data.drop(total_data.index[total_data['address'].isin(['Thạch Thất', 'Mê Linh'])])

# total_data[total_data['address'] == 'Mê Linh']

In [37]:
total_data

,index,address,bedroom,toilet,investor,acreage,price,lat,long,distance
0,0,Nam Từ Liêm,3.0,2.0,other,94.0,30.000000,21.012432,105.767395,9.030936
1,1,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,72.0,38.888889,21.019798,105.780708,7.527950
2,2,Cầu Giấy,2.0,2.0,tân hoàng minh group,82.0,34.146341,21.005806,105.794373,6.563944
3,3,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,69.0,29.000000,21.006529,105.753555,10.578458
4,4,Tây Hồ,4.0,2.0,tổng công ty đầu tư phát triển hạ tầng đô thị ...,148.0,35.810811,21.063967,105.805504,6.251858
...,...,...,...,...,...,...,...,...,...,...
29127,16395,Hoàng Mai,2.0,2.0,other,71.0,27.500000,20.956589,105.844887,8.087054
29128,16396,Hai Bà Trưng,2.0,2.0,tập đoàn vingroup,78.0,44.900000,20.996578,105.865196,3.831080
29129,16397,Long Biên,2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,NaN,21.035978,105.906296,5.644863
29130,16398,Long Biên,2.0,NaN,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.600000,21.062691,105.908333,6.914078


In [38]:
total_data = total_data.fillna(total_data.median())

In [39]:
total_data

,index,address,bedroom,toilet,investor,acreage,price,lat,long,distance
0,0,Nam Từ Liêm,3.0,2.0,other,94.0,30.000000,21.012432,105.767395,9.030936
1,1,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,72.0,38.888889,21.019798,105.780708,7.527950
2,2,Cầu Giấy,2.0,2.0,tân hoàng minh group,82.0,34.146341,21.005806,105.794373,6.563944
3,3,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,69.0,29.000000,21.006529,105.753555,10.578458
4,4,Tây Hồ,4.0,2.0,tổng công ty đầu tư phát triển hạ tầng đô thị ...,148.0,35.810811,21.063967,105.805504,6.251858
...,...,...,...,...,...,...,...,...,...,...
29127,16395,Hoàng Mai,2.0,2.0,other,71.0,27.500000,20.956589,105.844887,8.087054
29128,16396,Hai Bà Trưng,2.0,2.0,tập đoàn vingroup,78.0,44.900000,20.996578,105.865196,3.831080
29129,16397,Long Biên,2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,29.200000,21.035978,105.906296,5.644863
29130,16398,Long Biên,2.0,2.0,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.600000,21.062691,105.908333,6.914078


In [45]:
total_data = total_data.drop(['index'], axis=1)

In [46]:
total_data

,address,bedroom,toilet,investor,acreage,price,lat,long,distance
0,Nam Từ Liêm,3.0,2.0,other,94.0,30.000000,21.012432,105.767395,9.030936
1,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,72.0,38.888889,21.019798,105.780708,7.527950
2,Cầu Giấy,2.0,2.0,tân hoàng minh group,82.0,34.146341,21.005806,105.794373,6.563944
3,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,69.0,29.000000,21.006529,105.753555,10.578458
4,Tây Hồ,4.0,2.0,tổng công ty đầu tư phát triển hạ tầng đô thị ...,148.0,35.810811,21.063967,105.805504,6.251858
...,...,...,...,...,...,...,...,...,...
29127,Hoàng Mai,2.0,2.0,other,71.0,27.500000,20.956589,105.844887,8.087054
29128,Hai Bà Trưng,2.0,2.0,tập đoàn vingroup,78.0,44.900000,20.996578,105.865196,3.831080
29129,Long Biên,2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,29.200000,21.035978,105.906296,5.644863
29130,Long Biên,2.0,2.0,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.600000,21.062691,105.908333,6.914078


In [47]:
total_data.to_csv('total-data.csv', index=False)